# Load data and imports

In [3]:
import implicit
import random
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sparse

In [4]:
data_dir = 'training_mixpanel.txt'
with open(data_dir) as f:
    purchase_data = json.load(f)
data = [item['properties'] for item in purchase_data]
df_raw = pd.DataFrame(data)

In [5]:
df_raw.head()

,country,customer_id,description,invoice_date,invoice_no,product_id,quantity,unit_price
0,United Kingdom,17850,WHITE HANGING HEART T-LIGHT HOLDER,12/1/2010 8:26,536365,85123A,6,2.55
1,United Kingdom,17850,WHITE METAL LANTERN,12/1/2010 8:26,536365,71053,6,3.39
2,United Kingdom,17850,SET 7 BABUSHKA NESTING BOXES,12/1/2010 8:26,536365,22752,2,7.65
3,United Kingdom,17850,GLASS STAR FROSTED T-LIGHT HOLDER,12/1/2010 8:26,536365,21730,6,4.25
4,United Kingdom,17850,HAND WARMER UNION JACK,12/1/2010 8:28,536366,22633,6,1.85


# Preprocessing

## Remove some features and drop negative quantities

I'll treat this problem like a typical collaborative filtering problem. The only catch is that we don't have any explicit feedback, so we'll have to use the quantity of purchases (implicit feedback) instead. I perform the following preprocessing steps I perform to start:

- Remove all the features except for the one's I need for collaborative filtering. 
- Aggregate all the quantities purchased of each product by each customer. This will be the implicit feedback used for training the model.
- Set zero quantities to one. This assumes that the customer is still more likely to buy these products than one's they've never bought before.
- Remove negative quantities. This assumes that customers have a negative preference for products that we've seen them return more of than they've purchased.

In [6]:
# Only keep the customers, products, and quantity purchased.
df = df_raw[['customer_id','product_id','quantity']]
# This aggregates all the quantities purchased for each customer and product.
df = df.groupby(['customer_id','product_id']).sum().reset_index()
# Set zoer quantity sums to one.
df.quantity = df.quantity.apply(lambda x: 1 if x==0 else x)
# Remove negative quantities. 
df = df.query('quantity > 0').reset_index(drop=True)

In [7]:
df.head()

,customer_id,product_id,quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


## Encode customer and product ids

Next I encode the customer and product ids. All the models used below will create vector representations of each product and each customer. These vectors are stored either in a matrices rows or columns. By encoding the customers and products with unique integers, we can store their respective vectors at the matrix row/column which corresponds with their encoded name.

In [8]:
class Encoder(object):
    def __init__(self, col):
        self.names = col.unique()
        self.length = len(self.names)
        self.name2idx = {name:idx for idx,name in enumerate(self.names)}
    
    def __len__(self):
        return self.length
    
    def __call__(self,name):
        return self.name2idx[name]

In [9]:
enc_customers = Encoder(df['customer_id'])
enc_products = Encoder(df['product_id'])

In [10]:
n_customers = len(enc_customers)
n_products = len(enc_products)

In [11]:
df.customer_id = df.customer_id.apply(lambda x: enc_customers(x))
df.product_id = df.product_id.apply(lambda x: enc_products(x))

In [12]:
df.head()

,customer_id,product_id,quantity
0,0,0,1
1,1,1,24
2,1,2,36
3,1,3,6
4,1,4,40


# Split dataset

Next I split the data into a training and a validation set. The training set will be what the model sees during training, and then we'll evaluate the performance of the model on the validation set. This is done because we want a model which can predict well on new data which it hasn't seen before.

In [13]:
def get_val_idxs(idxs, pc):
    np.random.shuffle(idxs)
    val_idxs = idxs[0:int(pc*len(idxs))]
    return np.array(val_idxs)

In [14]:
# choose random subset for validation
val_percentage = 0.2
val_idxs = get_val_idxs(list(df.index), val_percentage)
mask = np.zeros(len(df),dtype=bool)
mask[val_idxs] = True

In [15]:
# create validation and training set
df_val = df[mask]
df_trn = df[~mask]

In [16]:
val_users = df_val.customer_id.unique()

Create a sparse matrix representation of the various datasets. 

The data that we have only includes a very small fraction of the entire set of customer/product combinations possible. For training our model, we'll use the fact that a particular user hasn't bought a particular product as an indication that they have a lower preference for that item, and are therefore less likely to buy it in the future. To represent this, we create a sparse matrix, where each row is a customer and each column is a product. This matrix will have zeros at all the customer/product locations that aren't included in the data. 

In [17]:
# sparse matrix for training data
sparse_trn = sparse.csr_matrix(
    (df_trn.quantity, (df_trn.customer_id, df_trn.product_id)), 
    shape=(n_customers, n_products))
# sparse matrix for validation data
sparse_val = sparse.csr_matrix(
    (df_val.quantity, (df_val.customer_id, df_val.product_id)), 
    shape=(n_customers, n_products))

# Evaluation Metric

The metric used for this problem is area under the ROC curve. A greater area under the curve means we are recommending items that end up being purchased near the top of the list of recommended products.

This metric only works with binary classification problem's, so we'll have to change our validation data to make this metric work. Instead of comparing to a validation dataset with quantities purchased for each item, we'll convert it into a binary preference matrix, where products that a customer has bought are represented with a one, and one's they haven't bought are represented with a zero.

In [139]:
# sparse matrix for validation using binary representation
sparse_val_bin = sparse.csr_matrix(
    (df_val.quantity.apply(lambda x: 1), (df_val.customer_id, df_val.product_id)), 
    shape=(n_customers, n_products))

In [18]:
from sklearn.metrics import roc_curve, auc
def mean_auc(pred, true, sparse_trn, val_users):
    scores = []
    # iterate through customers in the validation set
    for user in val_users: 
        # Only predict on user-item pairs which weren't seen during training.
        user_row_trn = sparse_trn[user,:].toarray().reshape(-1) 
        unseen_inds = np.where(user_row_trn == 0) 
        # Get user predicted and true purchase arrays
        y_pred = pred[user,:].toarray()[0][unseen_inds]
        y_true = true[user,:].toarray()[0][unseen_inds]
        # Calculate area under ROC
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        scores.append(auc(fpr, tpr)) 
    return np.mean(scores)

# Baseline

Before creating any models, it's always a good idea to create a baseline to measure model performance against. What I'll use as a baseline here is the mean quantity purchased over all users for each product. This roughly represents the quantity of each product that the average user will purchase. A recommender system which uses this baseline would just be ranking the items by popularity and would do so in the same way for each customer.

In [19]:
mean_quantity = np.array(sparse_trn.mean(axis = 0)).reshape(-1)
# create matrix which has a copy of the above vector at each customer row
pred_mat = np.full((n_customers,n_products),mean_quantity)
# convert matrix to sparse representation
sparse_pred = sparse.csr_matrix(pred_mat)

In [20]:
# AUROC
score = mean_auc(sparse_pred, sparse_val_bin, sparse_trn, val_users)
print('Area under ROC: {:0.4f}'.format(score))

Area under ROC: 0.7789


# ALS model

The first model tried is Alternating Least Squares. What the algorithm tries to do is find a vector for each customer and a vector for each product such that their inner product is that customers preference for that product. This is a common technique for collaborative filtering problems. What differentiates this algorithm is the way in which it optimizes the cost function (see this [paper](http://yifanhu.net/PUB/cf.pdf) by Hu, Koren, and Volinsky for the details). At each step in the optimization, you alternate between holding the customer-vectors fixed while re-computing the product-vectors and holding the product-vectors fixed while re-computing the customer-vectors. This approach allows for much faster computing of the customer and product vectors than many others, such as stochastic gradient descent.

In [21]:
class ALSmodel():
    def __init__(self, alpha=15, factors=10, regularization=0, iterations=1):
        self.alpha = alpha
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.model = implicit.als.AlternatingLeastSquares(factors=self.factors, 
                                                        regularization=self.regularization, 
                                                        iterations=self.iterations)
    def fit(self, sparse_mat):
        """Fit the model to the matrix provided
        
        Once the model has fit to the matrix,
        the computed user and item vectors are store
        
        Arguments:
        sparse_mat -- sparse matrix to fit to
        """
        self.model.fit(sparse_mat.T*self.alpha)
        self.user_vecs = sparse.csr_matrix(self.model.user_factors)
        self.item_vecs = sparse.csr_matrix(self.model.item_factors.T)
            
    def pred_mat(self):
        """ 
        Predict and return every user-item
        pair as a numpy array
        """
        pred = np.matmul(self.user_vecs.toarray(),self.item_vecs.toarray())
        return pred


In [22]:
# Create model
als_model = ALSmodel(alpha=15,factors=50,
                     regularization=0.1,
                     iterations=50)
# Fit to training data
als_model.fit(sparse_trn)

## Evaluate

In [23]:
# Predict on all customer-product pairs
pred_mat = als_model.pred_mat()

In [24]:
# Area under ROC
score = mean_auc(sparse.csr_matrix(pred_mat), sparse_val_bin, sparse_trn, val_users)
print('Area under ROC: {:0.4f}'.format(score))

Area under ROC: 0.8347


# SGD-nonzero model

This model also creates customer and product vectors whose dot product estimates the customer's preference for a product. But this approach randomly initializes the embeddings (customer/product vector) and then uses Stochastic Gradient Descent (SGD) to optimize them. This model will only learn on the customer-product examples included in the dataset, unlike ALS which assumes the customer preference for customer-product pairs it hasn't seen are zero. The hope here is that the data provided has enough information about the user and products such that useful representations can be found which can predict preferences on unseen customer-product pairs. The main motivation for only including customer-product pairs seen in the data is to reduce training time.

In [25]:
from keras.layers import Embedding, Reshape, Merge, Dot, Dense
from keras.models import Model, Sequential

Using TensorFlow backend.


In [26]:
class CFModel(Sequential):

    def __init__(self, n_customers, n_products, k_factors, **kwargs):
        self.n_customers = n_customers
        self.n_products = n_products
        self.user = Sequential()
        self.user.add(Embedding(n_customers, k_factors, input_length=1))
        self.user.add(Reshape((k_factors,)))
        self.item = Sequential()
        self.item.add(Embedding(n_products, k_factors, input_length=1))
        self.item.add(Reshape((k_factors,)))
        super(CFModel, self).__init__(**kwargs)
        self.add(Merge([self.user, self.item], mode='dot', dot_axes=1))
        self.add(Dense(1, activation='linear'))

    def pred(self, user_id, item_id):
        return self.predict([np.array([user_id]), np.array([item_id])])[0][0]
    
    def pred_vec(self, user_id):
        res = self.predict([np.array([user_id]*self.n_products),
                            np.arange(self.n_products)])
        return res.flatten()

In [40]:
k_factors = 50
emb_nonzero = CFModel(n_customers, n_products, k_factors)
emb_nonzero.compile(loss='mse', optimizer='rmsprop')

/home/taylor/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]


In [41]:
emb_nonzero.fit([df_trn.customer_id, df_trn.product_id], df_trn.quantity, epochs=1, batch_size=128, 
               validation_data=([df_val.customer_id, df_val.product_id], df_val.quantity))

Train on 170340 samples, validate on 42584 samples
Epoch 1/1
170340/170340 [==============================] - 7s 41us/step - loss: 9170.9965 - val_loss: 11697.4718


## Evaluate

In [36]:
# unravel sparse matrix and indices to feed into model
product_idxs = np.full((n_customers,n_products),np.arange(n_products))
customer_idxs = np.full((n_products,n_customers),np.arange(n_customers)).T
product_idxs = product_idxs.ravel()
customer_idxs = customer_idxs.ravel()

In [42]:
# predict on all customer-product pairs
pred = emb_nonzero.predict([customer_idxs, product_idxs])

In [43]:
# convert predictions to sparse matrix
pred_mat = sparse.csr_matrix(
    (pred.flatten(), (customer_idxs, product_idxs)), 
    shape=(n_customers, n_products))

In [44]:
# Area under ROC
score = mean_auc(pred_mat, sparse_val_bin, sparse_trn, val_users)
print('Area under ROC: {:0.4f}'.format(score))

Area under ROC: 0.7134


# SGD-full model

This model is the same as the one above, except we now include the customer-product pairs which aren't in the dataset, and assumes preference for these pairs are zero (same as ALS). Now the model will have all the information available to it that the ALS model has.

In [45]:
k_factors = 50
model = CFModel(n_customers, n_products, k_factors)
model.compile(loss='mse', optimizer='rmsprop')

/home/taylor/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]


In [ ]:
quantities = sparse_trn.toarray().ravel()

In [46]:
model.fit([customer_idxs, product_idxs], quantities, epochs=1, batch_size=128)

Epoch 1/1
15857430/15857430 [==============================] - 668s 42us/step - loss: 98.8710


## Evaluate

In [47]:
# predict on all customer-product pairs
pred = model.predict([customer_idxs, product_idxs])

In [48]:
# convert predictions to sparse matrix
pred_mat = sparse.csr_matrix(
    (pred.flatten(), (customer_idxs, product_idxs)), 
    shape=(n_customers, n_products))

In [49]:
# Area under ROC
score = mean_auc(pred_mat, sparse_val_bin, sparse_trn, val_users)
print('Area under ROC: {:0.4f}'.format(score))

Area under ROC: 0.6770


# Discussion

Model | AUROC 
--- | --- | 
ALS | 0.8347 | 
Baseline | 0.7789 |
SGD-nonzero | 0.7134 |
SGD-full | 0.6770 |


ALS is our winner! Followed by the baseline, SGD-nonzero, and SGD-full. 

ALS is an algorithm which was crafted specifically for these kinds of implicit rating problems. From the outset, it seemed likely that this model would outperform others, but I wanted to try out some embeddings learned through SGD to see how they held up. 

The first SGD optimized model (SGD-nonzero) I tried was trained only on the labeled data we have. It didn't include missing customer-product pairs as zero rated examples as the ALS algorithm does. This is an approach often used for problems with explicit ratings. The idea here is that, given the relative ratings between products and users, the model might still be able to learn embeddings which are useful for predicting relative preferences between products for a user. This approach stalled out in performance pretty quickly, which is reflected in it's low performance relative to the baseline.

SGD-full was trained on the entire sparse training matrix. So this one did receive the zero rating customer-product pairs to learn from. This model performed wost of all and took *way* longer to train than the ALS model. A single epoch took about 10 minutes, while the ALS was able to complete 50 iterations in seconds. I think this is performing worse that SGD-partial because the information gained from the non-zero customer-product pairs is being diluted by all the zero examples. To be fair, the model only trained for one epoch, and it might be the case that it's performance will steadly rise given enough time. But it might also be the case where the zero preference examples keep pushing the embeddings toward predictions of zero and never learns useful representations. In any case, given how long it takes to train, I think it's safe to say that for practical purposes ALS would be the way to go.

In [57]:
# save ALS model predictions
pred_mat = als_model.pred_mat()
np.save('user_profiles', pred_mat)